In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install rouge_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!gsutil cp -r gs://vietai_public/viT5/data/vietnews .
!gsutil cp -r gs://vietai_public/viT5/data/wikilingua .

Caught CTRL-C (signal 2) - exiting
^C
Copying gs://vietai_public/viT5/data/wikilingua/test.tsv...
Copying gs://vietai_public/viT5/data/wikilingua/train.tsv...
Caught CTRL-C (signal 2) - exiting
^C


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import json
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
import pandas as pd
import csv

In [ ]:
def save_pklFile(input_file, name):
  with open(f'/content/drive/MyDrive/ChatBot/data/{name}.pkl', 'wb') as f:

    # A new file will be created
    pickle.dump(input_file, f)

In [ ]:
def process_dataset(path):
  with open(path, 'r', encoding='utf-8') as df:
    df = json.load(df)
  train, test = train_test_split([x for x in df.keys()], test_size=0.2, random_state = 42)
  train = [df[x] for x in train]
  X_train = [x['document'] for x in train]
  y_train = [x['summary'] for x in train]
  save_pklFile(train, 'train')

  test = [df[x] for x in test]
  X_test = [x['document'] for x in test]
  y_test = [x['summary'] for x in test]
  save_pklFile(test, 'test')
  return X_train, X_test, y_train, y_test

In [ ]:
def load_pklFile(path):
  with open(path, 'rb') as f:

    # Call load method to deserialze
    return pickle.load(f, encoding='utf-8')

In [ ]:
def load_dataset():
  train = load_pklFile('/content/drive/MyDrive/ChatBot/data/train.pkl')
  X_train = [x['document'] for x in train]
  y_train = [x['summary'] for x in train]


  test = load_pklFile('/content/drive/MyDrive/ChatBot/data/test.pkl')
  X_test = [x['document'] for x in test]
  y_test = [x['summary'] for x in test]
  return X_train, X_test, y_train, y_test

In [ ]:
# X_train, X_test, y_train, y_test = process_dataset('/content/drive/MyDrive/ChatBot/data/data_DT_news.json')
X_train, X_test, y_train, y_test = load_dataset()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
# model.to('cuda')

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/ChatBot/tmp_1/checkpoint-7618")
model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=1024, truncation=True, padding=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=256, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
input_lines = []
label_lines = []

# task = 'wikilingua'
# train_file = 'train.tsv'

# with open(f'{task}/{train_file}') as file:
#   for line in file:
#     line = line.strip().split('\t')
#     input_lines.append(line[0] +'')
#     label_lines.append(line[1])




dict_obj = {'inputs': X_train, 'labels': y_train}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=8)


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=8):   0%|          | 0/2341 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword ar

In [ ]:
!pip install accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("/content/drive/MyDrive/ChatBot/tmp_1/",
                                      do_train=True,
                                      do_eval=False,
                                      num_train_epochs=20,
                                      learning_rate=1e-5,
                                      warmup_ratio=0.05,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,
                                      logging_dir='/content/drive/MyDrive/ChatBot/tmp_1/log',
                                      group_by_length=True,
                                      save_strategy="epoch",
                                      save_total_limit=3,
                                      #eval_steps=1,
                                      #evaluation_strategy="steps",
                                      # evaluation_strategy="no",
                                      fp16=True,
                                      )



# AdaFactor for ViT5-large models as it based on T5v1.1.
# See https://medium.com/the-artificial-impostor/paper-adafactor-adaptive-learning-rates-with-sublinear-memory-cost-a543abffa37
#
from transformers.optimization import Adafactor, AdafactorSchedule
optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

lr_scheduler = AdafactorSchedule(optimizer)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.274600
1000,0.243000
1500,0.232100
2000,0.223900
2500,0.211100


KeyboardInterrupt: 

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.279200
1000,0.252300
1500,0.230700
2000,0.209500
2500,0.185000
3000,0.169100
3500,0.152800
4000,0.134600
4500,0.124900
5000,0.119200


FailedPreconditionError: /content/drive/MyDrive/ChatBot/tmp_1/log/events.out.tfevents.1715865265.d6c5bc60f404.2784.0; Transport endpoint is not connected

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.279200
1000,0.252300
1500,0.230700
2000,0.209500
2500,0.185000
3000,0.169100
3500,0.152800
4000,0.134600
4500,0.124900
5000,0.119200


Step,Training Loss
500,0.279200
1000,0.252300
1500,0.230700
2000,0.209500
2500,0.185000
3000,0.169100
3500,0.152800
4000,0.134600
4500,0.124900
5000,0.119200


KeyboardInterrupt: 

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

<ipython-input-6-ffe5587f2e49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
input_lines = []
label_lines = []
# with open(f'{task}/test.tsv') as file:
#   for line in file:
#     line = line.strip().split('\t')
#     input = line[0]
#     input_lines.append(input +'')
#     label_lines.append(line[1])



input_lines  = X_test
label_lines = y_test
dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=10)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=10):   0%|          | 0/586 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

# Inference

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/ChatBot/tmp_1/checkpoint-7618")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
import torch
import numpy as np
metrics = load_metric('rouge')

max_target_length = 256
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=32)

predictions = []
references = []
for i, batch in enumerate(tqdm(dataloader)):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)


metrics.compute()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


  0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.5994873262947822, recall=0.5661410243981952, fmeasure=0.5722689746501982), mid=Score(precision=0.6083937931178482, recall=0.5764406546118552, fmeasure=0.57976059203447), high=Score(precision=0.6170563041924981, recall=0.5864970823070722, fmeasure=0.5863885959039816)),
 'rouge2': AggregateScore(low=Score(precision=0.2419191711068629, recall=0.22965437133937291, fmeasure=0.2312699535515112), mid=Score(precision=0.25179716372008343, recall=0.23952599600818758, fmeasure=0.24036528873036422), high=Score(precision=0.26165323758595455, recall=0.249274900306397, fmeasure=0.24956464271572648)),
 'rougeL': AggregateScore(low=Score(precision=0.3648138509117503, recall=0.34484721159939574, fmeasure=0.34803925955910386), mid=Score(precision=0.37299367594717403, recall=0.3529670046215674, fmeasure=0.35499203430012305), high=Score(precision=0.3816870842509065, recall=0.36152565881505727, fmeasure=0.36212564522023843)),
 'rougeLsum': AggregateScore(low=S

In [ ]:
[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.57976059203447},
 {'rouge2': 0.24036528873036422},
 {'rougeL': 0.35499203430012305},
 {'rougeLsum': 0.3553309771274256}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base-vietnews-summarization")
# model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base-vietnews-summarization")
# model.cuda()

sentence = "VietAI là tổ chức phi lợi nhuận với sứ mệnh ươm mầm tài năng về trí tuệ nhân tạo và xây dựng một cộng đồng các chuyên gia trong lĩnh vực trí tuệ nhân tạo đẳng cấp quốc tế tại Việt Nam."
sentence = sentence + "</s>"
encoding = tokenizer(sentence, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=256,
    early_stopping=True
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)


Tổ chức phi lợi nhuận VietAI hợp tác với các công ty phần mềm và trí tuệ nhân tạo hàng đầu tại Việt Nam, với mong muốn mang đến cho cộng đồng những chuyên gia, nhà khoa học, nhà khoa học và doanh nghiệp.


In [ ]:
def inference(sentence):
  sentence = sentence + "</s>"
  encoding = tokenizer(sentence, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
  outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=256,
      early_stopping=True
  )
  for output in outputs:
      line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
  return line

In [ ]:
txt = 'Cán bộ doanh nghiệp nêu thực tế nhiều lao động coi nhẹ tác phong làm việc, đi trễ, về sớm, hoặc đến đúng giờ điểm danh rồi làm việc riêng khiến năng suất không cao.Tại Diễn đàn nâng cao năng suất lao động quốc gia ngày 26/5, ông Mai Thiên Ân, Trưởng phòng sản xuất Công ty TNHH Intel Products Việt Nam, phản ánh thực tế tại nhiều nơi làm việc ở Việt Nam. Một bộ phận lao động không tuân thủ giờ giấc tác phong, có mặt ở cơ quan đúng giờ để điểm danh rồi ăn sáng, uống trà gây đình trệ công việc.Có doanh nghiệp lao động còn lấy sản phẩm bán ra ngoài khiến công ty bị thiệt hại hàng trăm triệu đồng, ảnh hưởng uy tín, mang tiếng văn hóa và con người Việt Nam. Trong sản xuất, một bộ phận lao động không tuân thủ quy trình khiến sản phẩm lỗi làm ảnh hưởng thương hiệu doanh nghiệp, lòng tin đối tác. Thậm chí, lao động không tuân thủ quy định dẫn đến nhiều vụ tai nạn. Ông Ân nhấn mạnh tác phong, kỷ luật làm việc còn bị xem nhẹ trong khi đây là yếu tố giúp tăng thu nhập lao động, nâng chất lượng sản phẩm, thương hiệu doanh nghiệp và góp phần nâng cao năng suất lao động quốc giaIntel là công ty đa quốc gia song nhân sự phần lớn là người Việt nên cũng gặp những vấn đề tương tự. Lãnh đạo cùng công đoàn đã thường xuyên đối thoại hàng tháng, hàng quý với người lao động để chỉ rõ tình huống thực tế rồi giải thích cho lao động hiểu, lắng nghe góp ý và phản hồi ngay ý kiến của họ. Doanh nghiệp này còn xây dựng quy trình và công cụ phản hồi ẩn danh để nhân viên đánh giá sự gương mẫu của quản lý.Để nâng cao năng suất lao động, ông Ân cho rằng cần đào tạo, chấn chỉnh tác phong con người từ khi còn đi học để dần hình thành "thói quen, nếp nghĩ, nếp làm". Học sinh cần được định hướng từ bậc phổ thông, đào tạo tác phong phù hợp ngành nghề đang theo học vì mỗi công việc cần ứng xử khác nhau."Quy chế tài chính cần cho phép công đoàn cơ sở có đủ nguồn lực trong chi tiêu cho việc đào tạo này, cũng như khen thưởng lẫn kỷ luật lao động", ông kiến nghị.Bà Phạm Thu Lan, Viện Công nhân Công đoàn, cho rằng tiền lương, thưởng, phúc lợi ảnh hưởng trực tiếp tới năng suất, động lực làm việc, sự toàn tâm của người lao động với công việc. Họ muốn gắn bó nhưng không thể ở lại mãi với công ty khi lương thấp. Đây là lý do tỷ lệ nhảy việc tại các ngành thâm dụng lao động luôn ở mức 8-12% mỗi tháng.Theo bà Lan, nhảy việc để tìm cơ hội mới, phát huy trình độ lẫn kỹ năng là bình thường, nhưng nếu thuần túy kiếm tìm mức lương cao hơn cho công việc tương tự là sự lãng phí. Ví dụ doanh nghiệp có 1.000 công nhân nhưng mỗi tháng 100 người liên tục ra vào thì sẽ rất tốn chi phí. Tiền bạc chi cho quảng cáo tuyển dụng, phỏng vấn, làm hồ sơ, đào tạo nhân viên, trong khi khoản này có thể tiết kiệm để đầu tư cho năng suất lao động.Việc học tập nâng cao năng suất theo bà Lan là cần thiết, song "có thực mới vực được đạo". Lao động còn vướng bận kiếm từng bữa cơm thì không thể ưu tiên học tập, chưa kể người thu nhập thấp không có tiền đầu tư học hành cho bản thân lẫn con cái. Trong khi lao động thiếu học tập nâng cao trình độ, Việt Nam sẽ không thể nâng cấp chuỗi giá trị, mãi nằm ở khâu gia công. Muốn tăng năng suất thì động lực là nâng mặt bằng tiền lương, thu nhập của người lao động lên. Bà Lan kiến nghị Chính phủ chỉ đạo Hội đồng Tiền lương quốc gia nghiên cứu và xác lập mức lương tối thiểu thỏa đáng. Đây là sàn an sinh cơ bản để lao động chi trả sinh hoạt phí, dự phòng bất trắc và tiết kiệm cho tương lai.Lắng nghe tham luận, Thủ tướng Phạm Minh Chính đề nghị các bộ ngành, Công đoàn tiếp thu để hoàn thiện chính sách rồi tổ chức thực hiện nhằm nâng cao năng suất lao động quốc gia. Đây là một trong những thước đo quan trọng để đánh giá, so sánh trình độ phát triển giữa các nước, các lĩnh vực, địa phương trong từng quốc gia, cũng là động lực để các nước đang phát triển vươn lên thoát bẫy thu nhập trung bình.Người đứng đầu Chính phủ điểm lại năng suất lao động đã tăng 2,7 lần từ năm 2011 đến năm 2023, mỗi người làm ra 70 triệu đồng tăng lên 188,7 triệu "là mức cao so với khu vực và đang thu hẹp dần khoảng cách với các nước". Thu nhập lao động từ đó cũng tăng lên, đạt 7,4 triệu đồng quý I năm nay.Song Thủ tướng cũng chỉ ra nhiều hạn chế lẫn thách thức như các cán bộ công đoàn, doanh nghiệp đã nêu. Cụ thể, tỷ lệ tăng năng suất lao động giai đoạn 2021-2023 vẫn ở mức thấp so với mục tiêu đề ra. Xét về giá trị tuyệt đối, năng suất lao động của Việt Nam vẫn thấp, năm 2022 chỉ tương đương 11,4% của Singapore, 24,7% của Hàn Quốc, 26,3% của Nhật Bản, 35,4% của Malaysia, 64,8% của Thái Lan, 79% của Indonesia, 94,5% của Philippines."Những yếu tố nền tảng cho tăng năng suất nhanh và bền vững như hạ tầng, chuyển dịch lao động từ nông thôn sang thành thị, từ phi chính thức sang chính thức chưa có bước đột phá", ông nói.Thủ tướng nêu rõ thời gian tới tình hình quốc tế phức tạp, khó lường tạo nên những "cơn gió ngược" cho sự phát triển. Bối cảnh đó đặt vấn đề nâng cao năng suất lao động là điều sống còn với đất nước. Muốn vậy cần tập trung vào "ba đẩy mạnh, ba tiên phong, ba bứt phá" để thúc đẩy năng suất lao động.Ba đẩy mạnh gồm hoàn thiện khung pháp lý để tạo môi trường đầu tư; huy động và sử dụng hiệu quả nguồn lực cho phát triển; đột phá chiến lược về nguồn nhân lực nhất là nhân lực chất lượng cao. Ba tiên phong chú trọng vào chuyển đổi số, các lĩnh vực mới tạo động lực cho nền kinh tế; hội nhập quốc tế để tận dụng cơ hội phát triển; thi đua tăng năng suất lao động. Ba bứt phá chọn khâu nguồn nhân lực; khoa học công nghệ và môi trường lao động.Thủ tướng yêu cầu các cấp ngành, địa phương, đặc biệt là Công đoàn Việt Nam chăm lo quyền và lợi ích chính đáng cho người lao động, đảm bảo đãi ngộ về tiền lương thưởng, các phúc lợi xã hội và triển khai nhanh chóng chương trình xây dựng một triệu căn nhà ở xã hội để người lao động có thêm chốn an cư'
inference(txt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'Thủ tướng chỉ ra nhiều hạn chế, thách thức của người lao động trong giai đoạn hiện nay như: tác phong làm việc, tác phong làm việc... đang ảnh hưởng trực tiếp tới năng suất lao động, góp phần nâng cao năng suất lao động.'

In [ ]:
def clean_txt(txt):
  ret = []
  ban = ['\xe0', '\xe1', '\xe2', '\xe3', '\xe4', '\xe5', '\xe6', '\xe7', '\xe8', '\xe9', '\u1ecd', '\u1ea1', '\u1ead', '\u1ea3', '\u1eeb', '\u1eed', '\u1ec7', '\xf4', '\u1ebf', '\u1ecb', '\u1ed1']
  for line in txt:
    for x in ban:
      if x in line:
        line = line.replace(x, '')
        print('done')
    ret.append(line)
  return ret

In [ ]:
reference = clean_txt(references)
prediction = clean_txt(predictions)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
don

In [ ]:
df_save = pd.DataFrame({'actuals' : references, 'predicts': predictions})

In [ ]:
df_save.to_csv('/content/result_inference.csv', encoding='ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\xe1' in position 4: ordinal not in range(128)